In [1]:
import numpy as np
from numpy.testing import assert_almost_equal

In [2]:
def sigmoid(x):
    return 1 / (1 + np.power(np.e, -x))

assert sigmoid(0) == 0.5
assert_almost_equal(sigmoid(-100), 0)
assert_almost_equal(sigmoid(100), 1)

def sigmoid_gradient(x):
    return sigmoid(x) * (1 - sigmoid(x))

assert_almost_equal(sigmoid_gradient(-100), 0)
assert_almost_equal(sigmoid_gradient(100), 0)

def forward_prop_nn(w1, b1, w2, b2, x):
    z1 = np.dot(w1, x) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = sigmoid(z2)
    return a2

xnor_w_1 = np.array([
    [-20, -20],          # Weights for "(NOT x[0]) AND (NOT x[1])" 
    [ 20,  20],          # Weights for "x[0] AND x[1]"
]) * 10
xnor_b_1 = np.array([
    [ 10],               # Bias for "(NOT x[0]) AND (NOT x[1])"
    [-30],               # Bias for "x[0] AND x[1]"
]) * 10
xnor_w_2 = np.array([
    [ 20,  20],          # Weights for "x[0] OR x[1]"
]) * 10
xnor_b_2 = np.array([
    [-10],               # Bias for "x[0] OR x[1]"
]) * 10

def xnor_nn(x0, x1):
    return forward_prop_nn(xnor_w_1, xnor_b_1, xnor_w_2, xnor_b_2, np.array(x).reshape(2, 1))[0][0]

xnor_truth_table = {
    (1, 1): 1.0,
    (0, 0): 1.0,
    (0, 1): 0.0,
    (1, 0): 0.0,
}

for x, y in xnor_truth_table.items():
    assert_almost_equal(xnor_nn(x[0], x[1]), y)

def train_nn(examples, iterations, learning_rate):
    m = len(examples)
    np.random.seed(1)
    w1 = np.random.rand(2, 2)
    b1 = np.zeros([2, 1])
    w2 = np.random.rand(1, 2)
    b2 = np.zeros([1, 1])
    for i in range(iterations):
        dw1 = np.zeros([2, 2])
        db1 = np.zeros([2, 1])
        dw2 = np.zeros([1, 2])
        db2 = np.zeros([1, 1])
        cost = np.zeros([1, 1])
        for x, y in examples:
            z1 = np.dot(w1, x) + b1
            a1 = sigmoid(z1)
            z2 = np.dot(w2, a1) + b2
            a2 = sigmoid(z2)

            cost += -y * np.log(a2) - (1 - y) * np.log(1 - a2)

            dz2 = a2 - y
            dw2 += np.dot(dz2, a1.T)
            db2 += dz2
            dz1 = np.dot(w2.T, dz2) * sigmoid_gradient(z1)
            dw1 += np.dot(dz1, x.T)
            db1 += dz1
        dw1 /= m
        db1 /= m
        dw2 /= m
        db2 /= m
        cost /= m
        w1 -= learning_rate * dw1
        b1 -= learning_rate * db1
        w2 -= learning_rate * dw2
        b2 -= learning_rate * db2
        if i % 100 == 0:
            print(f"cost at iteration {i}: {cost[0][0]}")
    return (w1, b1, w2, b2)

xnor_examples = [(np.array(x).reshape(2, 1), np.array([[y]])) for x, y in xnor_truth_table.items()]
w1, b1, w2, b2 = train_nn(xnor_examples, 5000, 1)

cost at iteration 0: 0.6960558242178866
cost at iteration 100: 0.6929867887355738
cost at iteration 200: 0.6922462617773397
cost at iteration 300: 0.6865890109283816
cost at iteration 400: 0.6374850000789614
cost at iteration 500: 0.5063366329094551
cost at iteration 600: 0.24494224360510392
cost at iteration 700: 0.0915469133772983
cost at iteration 800: 0.05089111333463649
cost at iteration 900: 0.034477698445246695
cost at iteration 1000: 0.025856753007441634
cost at iteration 1100: 0.020601465948381907
cost at iteration 1200: 0.017082139409163984
cost at iteration 1300: 0.014568584508727361
cost at iteration 1400: 0.012687400482552087
cost at iteration 1500: 0.011228647852513835
cost at iteration 1600: 0.010065565651741971
cost at iteration 1700: 0.009117240056884511
cost at iteration 1800: 0.008329668454476488
cost at iteration 1900: 0.007665468228175891
cost at iteration 2000: 0.007097960769063973
cost at iteration 2100: 0.00660760840370299
cost at iteration 2200: 0.0061797823890

In [3]:
for x, y in xnor_truth_table.items():
    y_hat = forward_prop_nn(w1, b1, w2, b2, np.array(x).reshape(2, 1))[0][0]
    assert_almost_equal(y_hat, y, decimal=2)
